In [3]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [4]:
# Convert the Excel files in the permit-data folder to CSV files

# Set the path to the folder with the Excel files
path = 'permit-data'

# Get a list of the Excel files in the folder
files = os.listdir(path)

# Loop through the files
for file in files:
    # Read the Excel file
    df = pd.read_excel(path + '/' + file)
    
    # Get the name of the file without the extension
    name = file.split('.')[0]
    
    # Write the CSV file
    df.to_csv('permit-data/' + name + '.csv', index = False)